In [1]:
%load_ext autoreload
%autoreload 2

This is a very simple example but shows how it is possible to structure your view/model in a hierarchical manner and use submodels to create reusable behavior, in this case the `Chart` widget relies only on the `Chart` state.

[model.py](model.py) shows the structure of the state.

Within [ui.py](ui.py), in particular, pay attention to `bindDropdown` copied below. Notice that we can read and write separate aspects of a state tree without knowing anything about the larger application through the use of submodels.

```python
bindDropdown(state.SubModel(m,lambda x: x.tickers),
             state.SubModel(m,lambda x: x.chart.selected_ticker)),

```

```python
def bindDropdown(options: state.protocols.Model[typing.List[state.T]],
                 target: state.protocols.Model[state.T]) -> Dropdown:
    dropdown = Dropdown()

    def update_options(options:typing.List[state.T]) -> None:
        dropdown.options = [str(s) for s in options]

    options.observe(lambda x: x, update_options)

    def update_target(newvalue:typing.Any) -> None:
        target.restore(options.state[newvalue['new']])

    dropdown.observe(update_target, 'index')
    return dropdown
```

The `Chart` widget knows nothing about how its data is generated, in this case, we can see in [model.py](model.py) that it is a simple GBM generator. One can imagine this loaded from a database or the web without modifying anything except how the data is loaded to its appropriate place in the tree.

In [3]:
from examples.notebooks.main import run

model,gui = run()

UI(children=(Dropdown(options=('TICK1', 'TICK2', 'TICK3'), value='TICK1'), Chart({
    'data': [{'close': arra…